In [2]:
# pour naviguer dans les répertoires
import os 
# pour manipuler les DataFrames
import pandas as pd
# pour ouvrir et inspecter les images
from PIL import Image

import numpy as np

# -----------------------------
# Parcours du dataset et collecte des métadonnées
# -----------------------------

# Répertoire de base où sont stockées les variantes d’images
base_dir = "../dataset/plantvillage/data/plantvillage dataset"
# liste qui accueillera un dict par image
data = []


# On parcourt les variantes/sous-dossiers "color", "grayscale", "segmented"
for variant in os.listdir(base_dir):
    variant_path = os.path.join(base_dir, variant)
    
    # Vérifie que c'est bien un dossier (pas un fichier)
    if os.path.isdir(variant_path):
        # Puis on parcourt chaque dossier d’espèce+disease
        for folder_name in os.listdir(variant_path):
            folder_path = os.path.join(variant_path, folder_name)
            
            if os.path.isdir(folder_path):
                # Extraire l’espèce et la maladie depuis le nom de dossier
                # Exemple : "Apple_scab__Powdery_mildew"
                if '__' in folder_name:
                    species, disease = folder_name.split('__', 1)
                else:
                    species, disease = folder_name, 'unknown'

                # Parcours de toutes les images dans ce dossier
                for image_name in os.listdir(folder_path):
                    image_path = os.path.join(folder_path, image_name)
                    
                    try:
                        # Ouverture sécurisée de l’image
                        with Image.open(image_path) as img:
                            # conversion systématique en RGB
                            img = img.convert('RGB')

                            # dimensions en pixels
                            width, height = img.size
                            # mode de couleur, ex RGB, grayscale,..# sera toujours 'RGB' après conversion
                            mode = img.mode  

                            # 2) Convertit en tableau NumPy et calcule mean/std par canal
                            arr = np.array(img)  # shape = (H, W, 3)
                            mean_R, mean_G, mean_B = arr.mean(axis=(0,1)).tolist()
                            std_R,  std_G,  std_B  = arr.std(axis=(0,1)).tolist()

                    except Exception as e:
                        # En cas d’erreur (fichier corrompu…), on l’affiche et on passe
                        print(f"Erreur avec {image_path} : {e}")
                        continue

                    # On stocke toutes les infos extraites dans un dict
                    data.append({
                        'image_path': image_path.replace('\\', '/'),
                        'variant': variant,  # color, grayscale ou segmented
                        'species': species,  # nom de l’espèce
                        'disease': disease, # type de maladie (ou 'unknown')
                        'width': width, # largeur
                        'height': height, # hauteur
                        'mode': mode,  # ex. 'RGB' 
                         # colonnes couleur globale
                        #'mean_R': mean_R, 'mean_G': mean_G, 'mean_B': mean_B,
                        #'std_R':  std_R,  'std_G':  std_G,  'std_B':  std_B
                    })

# Création du DataFrame pandas à partir de la liste de dicts
df = pd.DataFrame(data)
# Affichage des premières lignes pour vérifier la structure
print(df.head())
# Liste des colonnes disponibles dans le DF
print("\nColonnes disponibles :", df.columns.tolist())
# Statistiques descriptives sur les dimensions
print("\nDimensions des images :")
print(df[['width', 'height']].describe())

# Compte des différents modes d’image (RGB, L pour grayscale, etc.)
print("\nTypes d’image :", df['mode'].value_counts())
# Répartition du nombre d’images par variante (color, grayscale, segmented)
print("\nRépartition des variantes :", df['variant'].value_counts())

# Distribution des espèces
print(df['species'].value_counts())
# Distribution des maladies
print(df['disease'].value_counts())

KeyboardInterrupt: 

In [5]:
# vérif rapide
display(df.head())
print("Colonnes :", df.columns.tolist())

# le chargement du dataset complet et creation du dataframe prend 15 minutes
df.to_csv('../dataset/plantvillage/data/df_all_plantvillage_20250710.csv', index=False)

,image_path,variant,species,disease,width,height,mode
0,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
1,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
2,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
3,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
4,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB


Colonnes : ['image_path', 'variant', 'species', 'disease', 'width', 'height', 'mode']


In [2]:
# pour naviguer dans les répertoires
import os 
# pour manipuler les DataFrames
import pandas as pd
# pour ouvrir et inspecter les images
from PIL import Image

import numpy as np

# pour lancer plus vite : 0.3 s vs 15 minutes
df = pd.read_csv('../dataset/plantvillage/data/df_all_plantvillage_20250710.csv')
df.head()

,image_path,variant,species,disease,width,height,mode
0,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
1,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
2,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
3,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB
4,../dataset/plantvillage/data/plantvillage data...,segmented,Tomato,_Tomato_mosaic_virus,256,256,RGB


In [3]:
from sklearn.model_selection import train_test_split

# ---------------------------------------
# 2) Séparation X / y et split
# ---------------------------------------
X     = df.drop(columns=["species","image_path"], axis=1)
y     = df["species"]

# split avant tout fit
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=66, stratify=y
)


In [4]:
# ---------------------------------------
# 3) Sélection des colonnes num & cat
# ---------------------------------------
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()

In [5]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# ---------------------------------------
# 4) Pré-traitement séparé pour train et test
# ---------------------------------------
# 4a) scaler sur les var numériques
scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train[num_cols])
X_test_num_scaled  = scaler.transform(X_test[num_cols])

# remettre dans des DataFrame pour concat
X_train_num_df = pd.DataFrame(
    X_train_num_scaled,
    index=X_train.index,
    columns=num_cols
)
X_test_num_df = pd.DataFrame(
    X_test_num_scaled,
    index=X_test.index,
    columns=num_cols
)

# 4b) one-hot encoding des var catégorielles
encoder = OneHotEncoder(
    drop="first",
    handle_unknown="ignore",
    sparse_output=False
)
X_train_cat = encoder.fit_transform(X_train[cat_cols])
X_test_cat  = encoder.transform(  X_test[cat_cols])

# noms des nouvelles colonnes
cat_names = encoder.get_feature_names_out(cat_cols)

X_train_cat_df = pd.DataFrame(
    X_train_cat,
    index=X_train.index,
    columns=cat_names
)
X_test_cat_df = pd.DataFrame(
    X_test_cat,
    index=X_test.index,
    columns=cat_names
)

# concaténation finale
X_train_proc = pd.concat([X_train_num_df, X_train_cat_df], axis=1)
X_test_proc  = pd.concat([X_test_num_df,  X_test_cat_df],  axis=1)

In [6]:
from sklearn.svm import SVC
# ---------------------------------------
# 5) Entraînement du SVM
# ---------------------------------------
clf = SVC(kernel="rbf")
clf.fit(X_train_proc, y_train)

print("Score train :", clf.score(X_train_proc, y_train))
print("Score test :",  clf.score(X_test_proc,  y_test))

Score train : 0.6906899303317681
Score test : 0.6912595138718389


In [7]:
print("Avec l'ensemble du dataset plantvillage c'est mieux : ~ 0.69 en train et en test. Donc peu de sur-apprentissage. Mais perfectible")
print("tester avec cross-validation et GridSearch CV")

Avec l'ensemble du dataset plantvillage c'est mieux : ~ 0.69 en train et en test. Donc peu de sur-apprentissage. Mais perfectible
tester avec cross-validation et GridSearch CV


In [ ]:
# from sklearn.model_selection import GridSearchCV

# print(" Test avec cross-validation et GridSearch CV ")
# # définition de la grille d’hyperparamètres
# param_grid = {
#     "C":      [0.1, 1, 10, 100],
#     "gamma":  ["scale", 0.01, 0.1, 1],
#     "kernel": ["rbf", "poly"]
# }

# #GridSearchCV with 5‐fold CV
# grid = GridSearchCV(
#     estimator=SVC(),       # on passe directement le SVC non pré-configuré
#     param_grid=param_grid,
#     cv=5,               # 5-fold cross‐validation interne
#     scoring="accuracy",
#     n_jobs=-1
# )

# #fit sur tout le train pré-traité
# grid.fit(X_train_proc, y_train)

# print("Meilleurs params :", grid.best_params_)
# print("Meilleur score CV :", grid.best_score_)
# print("Score test final :", grid.score(X_test, y_test))

 Test avec cross-validation et GridSearch CV 


In [1]:
print("j'y arrive pas.. la machine s'arrete ca dure trop longtemps")

j'y arrive pas.. la machine s'arrete ca dure trop longtemps
